In [ ]:
from mxnet import gluon
from mxnet import ndarray as nd

获取数据

In [ ]:
def transform(data,label):
    return data.astype('float32')/255,label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True,root="E:/dataset/fashion-mnist",transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False,root="E:/dataset/fashion-mnist",transform=transform)

In [ ]:
data,label = mnist_train[0]
('example shape',data.shape,'label',label)

In [ ]:
import matplotlib.pyplot as plt

def show_images(images):
    n = images.shape[0]
    _,figs = plt.subplots(1,n,figsize=(15,15))
    for i in range(n):
        figs[i].imshow(images[i].reshape((28,28)).asnumpy())
        figs[i].axes.get_xaxis().set_visible(False)
        figs[i].axes.get_yaxis().set_visible(False)
    plt.show()

def get_text_labels(label):
    text_labels = [
        't-shirt','trouser','pullover','dress','coat',
        'sandal','shirt','sneaker','bag','ankle boot'
    ]
    return [text_labels[int(i)] for i in label]

data,label = mnist_train[0:9]
show_images(data)
print(get_text_labels(label))

数据读取

In [ ]:
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train,batch_size,shuffle=True)
test_data = gluon.data.DataLoader(mnist_test,batch_size,shuffle=False)

初始化模型参数

In [ ]:
num_inputs = 784
num_outputs = 10

W = nd.random_normal(shape=(num_inputs,num_outputs))
b = nd.random_normal(shape=num_outputs)

params = [W,b]

In [ ]:
for param in params:
    param.attach_grad()

In [ ]:
from mxnet import nd 
def softmax(X):
    exp = nd.exp(X)
    #假设exp是矩阵，这里对行进行求和，并要求保留axis 1,
    #就是返回(nrows,1)形状的矩阵
    partition = exp.sum(axis=1,keepdims=True)
    return exp/partition

In [ ]:
X = nd.random_normal(shape=(2,5))
X_prob = softmax(X)
print(X)
print(X_prob)
print(X_prob.sum(axis=1))

In [ ]:
def net(X):
    return softmax(nd.dot(X.reshape((-1,num_inputs)),W)+b)

In [ ]:
def cross_entropy(yhat,y):
    return -nd.pick(yhat,y).log()

计算精度

In [ ]:
def accuracy(output,label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

In [ ]:
def evaluate_accuracy(data_iterator,net):
    acc = 0.
    for data,label in data_iterator:
        output = net(data)
        acc += accuracy(output,label)
    return acc/len(data_iterator)

In [ ]:
evaluate_accuracy(test_data,net)

训练

In [ ]:
import sys
sys.path.append('..')
from utils import SGD
from mxnet import autograd

learning_rate = .1

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data,label in train_data:
        with autograd.record():
            output = net(data)
            loss = cross_entropy(output,label)
        loss.backward()
        #将梯度做平均，这样学习率会对batch size 不那么敏感
        SGD(params,learning_rate/batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output,label)

    test_acc = evaluate_accuracy(test_data,net)
    print("Epoch %d Loss %f Train acc %f Test acc%f"%(epoch,train_loss/len(train_data),train_acc/len(train_data),
    test_acc))



预测

In [ ]:
data,label = mnist_test[0:9]
show_images(data)
print('true label')
print(get_text_labels(label))

predicted_labels = net(data).argmax(axis=1)
print('predicted labels')
print(get_text_labels(predicted_labels.asnumpy()))